# 1. Data and python environment

## 1.1. Copernicus Marine Service Hindcast marine currents velocity dataset

Using CMEMS access it was downloaded the dataset with forcing variables (Marine currents)
Horizontal Surface Velocity (2D) - Hourly Mean 
The data was downloaded using the browser access to Copernicus Marine Services, product id: [med-cmcc-cur-rean-h_202012]

file is stored locally in the same folder of this notebook [med-cmcc-cur-rean-h_1751886654126.nc](med-cmcc-cur-rean-h_1751886654126.nc)



In [1]:
# check dataset downloaded
import xarray as xr
DS = xr.open_dataset('med-cmcc-cur-rean-h_1751886654126.nc')
DS

<xarray.Dataset> Size: 51MB
Dimensions:    (time: 505, latitude: 121, longitude: 104)
Coordinates:
  * time       (time) datetime64[ns] 4kB 2021-08-20T23:00:00 ... 2021-09-10T2...
  * latitude   (latitude) float32 484B 32.23 32.27 32.31 ... 37.15 37.19 37.23
  * longitude  (longitude) float32 416B 32.0 32.04 32.08 ... 36.21 36.25 36.29
Data variables:
    uo         (time, latitude, longitude) float32 25MB ...
    vo         (time, latitude, longitude) float32 25MB ...
Attributes:
    Conventions:       CF-1.11
    title:             Horizontal Surface Velocity (2D) - Hourly Mean
    institution:       Centro Euro-Mediterraneo sui Cambiamenti Climatici - C...
    source:            MFS E3R1
    contact:           servicedesk.cmems@mercator-ocean.eu
    references:        Please check in CMEMS catalogue the INFO section for p...
    comment:           Please check in CMEMS catalogue the INFO section for p...
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  MEDSEA_MULTIYEAR_PHY_006_004
    subset:datasetId:  med-cmcc-cur-rean-h_202012
    subset:date:       2025-07-07T11:10:54.137Z

## 1.2 Simulation data

To conduct the validation we use a multi-source release scenario.
It is a simulation of the condition of oil slik reported by SAR data acquired for the study conducted by R.M. Abou Samra and R.R. Ali (DOI: 10.1016/j.marpolbul.2023.115887), at the date of 2021-08-29 16:36.

To use `heco.run()` function in a multi-source condition we use a multitude of input file, with a variation of values only for source location.

Template yaml file is the following
```yaml
input:
  dataset_file_name: med-cmcc-cur-rean-h_1751886654126.nc
  lat0: #variable
  lon0: #variable
  sim_diffusion_coeff: 20.0
  sim_duration_h: 20.0 # from 29-Aug-2021 16:36 to 30-Aug-2021 10:58
  sim_particles: 100.0
  sim_timedelta_s: 3600.0
  spill_release_duration_h: 1.0
  time0: '2021-08-29 16:36:07'
  volume_spilled_m3: 100.0

```


In [2]:
# create an array of points coordinates
# import point list from geojson file

import json
with open('virtual_points_oil_spill-test2/virtual_origin_oil_spill_points_2.geojson', 'r') as f:
    geojson_data = f.read()
    geojson_points = json.loads(geojson_data)

# create a template yaml file
template = """input:
  dataset_file_name: med-cmcc-cur-rean-h_1751886654126.nc
  lat0: {lat}
  lon0: {lon}
  sim_diffusion_coeff: 20.0
  sim_duration_h: 20.0
  sim_particles: 100.0
  sim_timedelta_s: 3600.0
  spill_release_duration_h: 1.0
  time0: '2021-08-29 16:36:07'
  volume_spilled_m3: 100.0   
"""
# create a yaml file for each point
id = 0
for point in geojson_points['features']:
    lat = point['geometry']['coordinates'][1]
    lon = point['geometry']['coordinates'][0]
    yaml_content = template.format(lat=lat, lon=lon)

    # create an incremental id
    id += 1

    # write to a yaml file
    file_name = f"virtual_points_oil_spill-test2/virtual_origin_oil_spill_{id}.yaml"
    with open(file_name, 'w') as yaml_file:
        yaml_file.write(yaml_content)
    
    print(f"Created {file_name} for point ({lat}, {lon})")


Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_1.yaml for point (35.35416718134785, 35.12500110973005)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_2.yaml for point (35.39583387828989, 35.25000112706958)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_3.yaml for point (35.35416718134785, 35.08333443728354)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_4.yaml for point (35.35416718134785, 35.16666778217656)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_5.yaml for point (35.687500756884205, 34.95833441994401)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_6.yaml for point (35.687500756884205, 34.9166677474975)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_7.yaml for point (35.604167363000116, 34.87500107505099)
Created virtual_points_oil_spill-test2/virtual_origin_oil_spill_8.yaml for point (35.687500756884205, 35.00000109239052)
Created virtual_points_oil_spill-test

![img](virtual_points_oil_spill/figure7b-Abou_Samra&Ali_2024.png)

# 2. Run Simulation

Using a custom function we will run a Lagrangian model that simulate the emission of spill started from data retrieved by SAR in 2021-08-19

In [3]:
import heco
import geopandas as gpd
import glob

# create folder if not exists
import os
if not os.path.exists('results'):
    os.makedirs('results')

id = 0
for yaml_file in sorted(glob.glob('virtual_points_oil_spill-test2/virtual_origin_oil_spill_*.yaml')):
    id += 1
    print(f"Running HECO for {yaml_file}")
    output = heco.run(yaml_file, 'uo', 'vo') # 'uo' for x-forcing , 'vo' for y-forcing
    # save result to csv
    output.to_csv(f"results/test2/heco-val-test2_results_{id}.csv", index=False)
    # create gejson file
    gdf = gpd.GeoDataFrame(output, geometry=gpd.points_from_xy(output.lon, output.lat))
    gdf.crs = "EPSG:4326"
    gdf.to_file(f"results/test2/heco-val-test2_results_{id}.geojson", driver='GeoJSON')



Running HECO for virtual_points_oil_spill-test2/virtual_origin_oil_spill_1.yaml
Dataset med-cmcc-cur-rean-h_1751886654126.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill-test2/virtual_origin_oil_spill_10.yaml
Dataset med-cmcc-cur-rean-h_1751886654126.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill-test2/virtual_origin_oil_spill_11.yaml
Dataset med-cmcc-cur-rean-h_1751886654126.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill-test2/virtual_origin_oil_spill_12.yaml
Dataset med-cmcc-cur-rean-h_1751886654126.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill-test2/virtual_origin_oil_spill_13.yaml
Data

## 3. Aggregate results

Now aggregate results tables in ones and unique geojson

In [4]:
# merge all csv file in one pandas dataframe
import pandas as pd
results_global = pd.DataFrame()

id = 0
for csv_file in sorted(glob.glob('results/test2/heco-val-test2_results_*.csv')):
    id += 1
    # open csv and append in 
    df = pd.read_csv(csv_file)
    # insert colum with id of virtual origin spill id
    df.insert(0, 'virtual_origin_id', id)
    results_global = pd.concat([results_global, df], ignore_index=True)

In [5]:
# now create a geojson

gdf = gpd.GeoDataFrame(results_global, geometry=gpd.points_from_xy(results_global.lon, results_global.lat))
gdf.crs = "EPSG:4326"
gdf.to_file('results/test2/heco-val-test2_results_global.geojson', driver='GeoJSON')

## 3.3 Webmap and convex polygons

Using a custom function called `create_webmap`it is possible to generate an html that contain a LeafLet webmap and various EMODnet WMS Layers pre-configured.

It is also possible to pass the parameter `savepolygons = True` to convert points geometries in a convex-hull polygon that contain all particle for each time instant. This is a more efficent way to display and share the dispersions result across GIS user and map visualizers. The file will be saved in the default name `heco-polygons.geojson`

In [7]:
# export webmap

heco.create_webmap(
    HECOpoint_output_gdf_path = 'results/test2/heco-val-test2_results_global.geojson',
    EMODnetLayers = True,
    settingsFile_path = 'virtual_points_oil_spill-test2/virtual_origin_oil_spill_1.yaml',
    output_path = 'results/test2/heco-test2_map_global.html',
    savepolygons = True
)